# Lunar Lander: Simulation = Python + Math

## 3. Build a simulation framework


### 3.1 Running from start to finish
All this development and testing would be easier if lander could run a simulation from **start** to **finish**, by itself:
- *start* is whatever initial conditions exist (like altitude or velocity)
- *finish* is when the lander hits the surface of the moon
- a *go* function runs the simulation to completion
- at the end, the *go* function prints and graph the results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

class LunarLander:                   
    
    g = -2.0

    def __init__(self, mass = 120.0, fuel = 10.0, altitude = 0.0, velocity = 0.0):
        self.mass = mass     
        self.fuel = fuel
        self.altitude = altitude
        self.velocity = velocity
        self.simulation_completed = False       # this variable tracks whether a simulation is completed
        self.altitudes = []                     # this list tracks altitude over time
        self.destroyed = False

    def one_second(self): 
        self.velocity += LunarLander.g
        self.altitude += self.velocity
        self.altitudes.append(self.altitude)    # each altitude is recorded in the list 'altitudes'
        
        if (self.altitude <= 0.0):
            self.altitude = 0.0
            if abs(self.velocity) > 2.0:
                self.destroyed |= True
            self.simulation_completed = True;   # the simulation is completed when the lander hits the surface
        return self.altitude
    
    def fire_engine(self, fuel):
        if not self.destroyed:
            fuel = min(fuel, self.fuel)
            acceleration = fuel * 100.0
            self.velocity += acceleration
            self.fuel -= fuel

    def go(self):
        while not self.simulation_completed:                  # while the simulation is not complete...
            self.one_second()                                 # ...advance one second
        plt.plot(self.altitudes)                              # plot altitude as a function of time
        print('Impact velocity =', self.velocity, 'm/s^2')    # print final velocity
        if self.destroyed:                                    # note whether lander is destroyed
            print('Destroyed on impact!')
        

# here is a lander with an initial velocity of 100:      
LunarLander(velocity=100.0).go()

### 3.2 Inserting an engine control function

In Python, it's easy to pass a function to another function... just use the first function's name as a parameter when calling the second function. Using that approach, you can build a simulation framework, then 'plug in' an external function to provide a control mechanism.

That approach allows you to swap in various control mechanisms, and experiment with the results.

Here is a tiny example of passing functions in Python:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


# this function calls 'f' repeatedly, then adds the result to a graph

def graph(f):
    y = [f(x) for x in range(-10,10)]   # call 'f' for various values of x, regardless of what 'f' does...
    plt.plot(y)                         # add the results to a graph

    
# here are some functions...

def a(x):                               # some sort of parabola...
    return x*x / 100

def b(x):                               # something shaped like an 'S'
    return 2 / (1 + 2**(-x)) - 1

def c(x):
    return (x**3 - 100 * x) / 1000      # something like an 'S' that fell over backwards


# let's pass each function to draw_graph

graph(a)
graph(b)
graph(c)
plt.show()

With that in mind, let's insert an **engine control function** into our lander:
- the control function takes a LunarLander as its sole parameter
- the control function returns the amount of fuel to burn for the current one-second time step
- depending on the goal, different control functions are required
- any given control funtion can be as complex or as simple as needed

Here is a LunarLander that accepts an **engine control function**:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

class LunarLander:                   
    
    g = -2.0

    def __init__(self, mass = 120.0, fuel = 10.0, altitude = 0.0, velocity = 0.0):
        self.mass = mass     
        self.fuel = fuel
        self.altitude = altitude
        self.velocity = velocity
        self.simulation_completed = False 
        self.altitudes = []
        self.destroyed = False

    def one_second(self, engine_control_function):          # allow for an engine control function
        
        if not engine_control_function == None:             # check if a control function is present...
            self.fire_engine(engine_control_function(self)) # if so, pass it this lander, then fire the engine.
        
        self.velocity += LunarLander.g
        self.altitude += self.velocity
        self.altitudes.append(self.altitude)
        
        if (self.altitude <= 0.0):
            self.altitude = 0.0
            if abs(self.velocity) > 2.0:
                self.destroyed |= True
            self.simulation_completed = True;
        return self.altitude
    
    def fire_engine(self, fuel):
        if not self.destroyed:
            fuel = min(fuel, self.fuel)
            acceleration = fuel * 100.0
            self.velocity += acceleration
            self.fuel -= fuel

    def go(self, control_function=None):
        while not self.simulation_completed:
            self.one_second(control_function)
        plt.plot(self.altitudes)
        print('Impact velocity =', self.velocity, 'm/s^2')
        if self.destroyed:
            print('Destroyed on impact!')

### 3.2.1 Here is an example of an engine control function that does a poor job of seeking a specified altitude, then runs out of fuel and crashes:

In [ ]:
# here is a (silly) engine control function
def ecf(lander):
    if lander.altitude < 100.0:
        return lander.fuel * 0.01 - lander.velocity * 0.005
    else:
        return 0.0

# run the simulation using 'ecf' as the engine control function     
LunarLander().go(ecf)

### 3.2.2 Here is an example of an engine control function that boosts the lander upward in a series of steps:

In [ ]:
# here is a different engine control function

def steps(lander):
    if lander.altitude < 100.0:
        return 2.0;
    elif lander.velocity < 0.0:
        return 1.0
    else:
        return 0.0

# run the simulation using 'ecf' as the engine control function     
LunarLander().go(steps)

## Exercises



Create an engine control function that causes a lander to attain the highest altitude that you can, while still landing safely. It's OK to use trial and error, or math, or both.

In [ ]:
# Your control function goes here...

# Launch the simulation like thos:
LunarLander().go(your_function_name)